In [4]:
# Грузим библиотечки

import numpy as np
import pandas as pd

# !pip install category_encoders

import catboost as ctb
import category_encoders as ce

# !pip install shap

import shap

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

In [5]:
# и одну функцю оценки модели:

def evaluate_model(model, X_train, y_train, X_test, y_test):
    """
    Обучить и оценить модель.
    """
    model.fit(X_train, y_train, classifier__verbose=False)
    y_pred = model.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='binary')
    rec = recall_score(y_test, y_pred, average='binary')
    
    return (model, {'f1': [f1], 'roc-auc': [roc], 'precision': [prec], 'recall': [rec]})


#### Задание 1: взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

Пусть будет этот: https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists

In [6]:
df = pd.read_csv("aug_train.csv")
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


#### Задание 2: сделать обзорный анализ выбранного датасета

In [7]:
# Посмотрим общие данные датасета:
df.shape

(19158, 14)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [9]:
df.describe()

,enrollee_id,city_development_index,training_hours,target
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


In [10]:
# Конвертируем таргет в int, т.к. у нас задача классификации, а не регрессии:

df['target'] = df['target'].astype(int)

In [11]:
# Что с балансом таргета?

df['target'].value_counts()

0    14381
1     4777
Name: target, dtype: int64

Есть небольшой дисбаланс, но всё не так плохо

In [12]:
# Посмотрим кардинальность категориальных (object) фичей:

for feat in df.select_dtypes('object').columns:
    print(f'Кардинальность {feat}: {df[feat].nunique()}')

Кардинальность city: 123
Кардинальность gender: 3
Кардинальность relevent_experience: 2
Кардинальность enrolled_university: 3
Кардинальность education_level: 5
Кардинальность major_discipline: 6
Кардинальность experience: 22
Кардинальность company_size: 8
Кардинальность company_type: 6
Кардинальность last_new_job: 6


Можно попробовать сделать one-hot encoding для признаков с кардинальностью < 10.

Посмотрим, какие категории есть у признака experience.

In [13]:
df['experience'].value_counts()

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: experience, dtype: int64

Это просто кол-во лет опыта работы. Можно заменить ">20" на 21, "<1" на 0.

Признак city - явно просто города, в которых работают сотрудники. 
Чтобы закодировать этот признак используем CatBoostEncoder из библиотеки category_encoders.

Сделаем кодирование признаков в следующем задании.

#### Задание 3: сделать feature engineering

In [14]:
# Выберем числовые и категориальные признаки:

num_feats = df.select_dtypes('number').drop(columns='target').columns

cat_feats = df.select_dtypes('object').columns

In [15]:
# Не будем брать ненужный признак enrollee_id, т.к. это просто числовой ID сотрудника:

num_feats = num_feats.drop('enrollee_id')

Заполняем пропуски с помощью SimpleImputer для всех категориальных признаков (на случай отсутствия будущих данных).

Заменяем все пропущенные значения категориальных признаков на самое частое значение (моду), числовых - на медиану.

In [16]:
num_imputer = Pipeline([
            ('imputer', SimpleImputer(strategy='median'))
        ])

cat_imputer = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent'))
        ])

imputers = ColumnTransformer([
    ('num_imputer', num_imputer, num_feats),
    ('cat_imputer', cat_imputer, cat_feats),
])

In [17]:
# Числовые признаки берем как они есть:

class NumberTaker(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X

In [18]:
num_transformer = Pipeline([
    ('nums', NumberTaker())
])

Кодируем категориальные признаки:

In [19]:
# One-hot encoding для всех признаков, кроме experience, city (кардинальность < 10):

cat_transformer = Pipeline([
    ('ohe', OneHotEncoder(drop='first', sparse=False))
])

In [20]:
# Трансформер кодирования признака experience:

class ExperienceTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):  # , y=None
        X = X.copy()
        X.loc[X[self.column_name] == '<1', self.column_name] = 0
        X.loc[X[self.column_name] == '>20', self.column_name] = 21
        X = X.astype(int)
        return X

In [21]:
experience_transformer = Pipeline([
    ('experience_transform', ExperienceTransformer('experience'))
])

In [22]:
# Кодируем признак city с помощью CatBoostEncoder:

city_transformer = Pipeline([
    ('city_transform', ce.cat_boost.CatBoostEncoder())
])

In [23]:
# Собираем все трансформации в один пайплайн:

transformers = ColumnTransformer([
    ('num_transformer', num_transformer, num_feats),
    ('cat_transformer', cat_transformer, cat_feats.drop(['experience', 'city'])),
    ('experience_transformer', experience_transformer, ['experience']),
    ('city_transformer', city_transformer, ['city']),
])

Собираем вместе заполнение пропусков и трансформации.

SimpleImputer возвращает данные в numpy.ndarray, поэтому нужно снова создать датафрейм, чтобы выбирать по столбцам признаков.

In [24]:
class NumpyToDataFrame(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(X, columns=self.column_names)

In [25]:
preprocessing = Pipeline([
    ('imputers', imputers),
    ('numpy_to_df', NumpyToDataFrame(num_feats.tolist() + cat_feats.tolist())),
    ('transforms', transformers)
])

#### Задание 4: обучить любой классификатор (какой вам нравится)

In [26]:
# Разделим данные на трейн и тест:

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']), df['target'], test_size=0.2, random_state=42)

In [27]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('classifier', ctb.CatBoostClassifier()), # cat_features=['experience', 'city'])),
])

In [28]:
model, model_metrics = evaluate_model(pipeline,
                                      X_train,
                                      y_train,
                                      X_test,
                                      y_test)

In [29]:
metrics = pd.DataFrame(model_metrics)

In [30]:
metrics

,f1,roc-auc,precision,recall
0,0.386555,0.610236,0.579832,0.289916


#### Задание 5: разобраться с SHAP и построить важности признаков для:
#### - всего тестового набора данных (summary_plot - дать интерпретацию)
#### - для топ 10%

Shap не поддерживает sklearn Pipeline, поэтому обработаем данные отдельно, чтобы подать их в SHAP Explainer.

In [33]:
class DataProcessor:
    def __init__(self):
        self.data_imputers = ColumnTransformer([
            ('num_imputer', num_imputer, num_feats),
            ('cat_imputer', cat_imputer, cat_feats),
        ])
        self.city_ctb_encoder = ce.cat_boost.CatBoostEncoder(return_df=True)
        self.ohe_encoder = OneHotEncoder(drop='first', sparse=False)
    
    def fit(self, X, y=None):
        # Imputers
        self.data_imputers.fit(X, y)
        # Encode city
        self.city_ctb_encoder.fit(X['city'], y)
        # One-hot encoder
        self.ohe_encoder.fit(X[cat_feats.drop(['experience', 'city'])], X, y)
        
        return self
    
    def transform(self, X, y=None):
        X_trans = X.drop(columns=['enrollee_id'])
        # Imputers
        X_trans = pd.DataFrame(self.data_imputers.transform(X), columns=num_feats.tolist() + cat_feats.tolist())
        # Put numeric columns back to numeric
        X_trans[['city_development_index',
                 'training_hours']] = X_trans[['city_development_index', 'training_hours']].astype(float)
        # Encode experience
        X_trans.loc[X_trans['experience'] == '<1', 'experience'] = 0
        X_trans.loc[X_trans['experience'] == '>20', 'experience'] = 21
        X_trans['experience'] = X_trans['experience'].astype(int)
        # Encode city
        X_trans['city'] = self.city_ctb_encoder.transform(X_trans['city'])
        # One-hot for low-cardinality features
        X_trans = pd.get_dummies(X_trans, drop_first=True)

#         ohe_feats = cat_feats.drop(['experience', 'city']).tolist()
#         X_trans = pd.concat([
#             X_trans.drop(columns=ohe_feats),
#             pd.DataFrame(self.ohe_encoder.transform(X_trans[ohe_feats])),
#         ], axis=1)
        
        return X_trans
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [34]:
shap_data_processor = DataProcessor().fit(X_train, y_train)
X_test_trans = shap_data_processor.transform(X_test)

ValueError: Input contains NaN

In [35]:
X_test_trans.info()

NameError: name 'X_test_trans' is not defined